<a href="https://colab.research.google.com/github/wsimpso1/recent_projects/blob/main/NLP_Text_Summarization_War_In_Ukraine/Part_2_text_sum_ukraine_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic Twitter Event Summarization: Making Sense of War in Ukraine
---
**Aishwarya Kale and William Simpson** 

Dept. of Computer Science & Electrical Engineering, University of Maryland, Baltimore County 

DATA 690: Introduction to Natural Language Processing 

Dr. Antonio Diana 

May 6, 2022

**Scroll down to see the beginning of Part 2**

# Install Necessary Libraries

In [ ]:
!pip install -q bertopic
!pip install -q python-louvain
!pip install -q -U sentence-transformers
!pip install -q transformers
!pip install -q rouge
!pip install -q emoji
!pip install -q pandarallel 
!pip install -q sumy
!pip install -q textrank4zh
!pip install -q textstat

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import re
import numpy as np
import nltk
import matplotlib.pyplot as plt
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt') 
from datetime import datetime, timedelta
import ast

# ner
from spacy import displacy
import emoji
import string

# topic modeling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from bertopic import BERTopic

# clustering
from sklearn.metrics.pairwise import linear_kernel
from tqdm.notebook import tnrange
import networkx as nx
import community.community_louvain as community_louvain
from sentence_transformers import SentenceTransformer, util
import umap
import hdbscan
from bertopic import BERTopic
from gensim.summarization import keywords
from wordcloud import WordCloud

# sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob as tb
nltk.download('vader_lexicon')
from pandarallel import pandarallel

# summarization
from transformers import pipeline
from rouge import Rouge 
from statistics import mean
import seaborn as sns
import math
from collections import Counter
from nltk.probability import FreqDist
from sentence_transformers.util import cos_sim as cos_sim_
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
import textwrap
from gensim.summarization import summarize
from spacy.lang.en.stop_words import STOP_WORDS
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from string import punctuation
from heapq import nlargest
import sys
from textrank4zh import TextRank4Sentence
import textstat

# settings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action="ignore", category=DeprecationWarning)
import multiprocessing
# check number of cores available
print('\nNUMBER OF CPU CORES:', multiprocessing.cpu_count())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!

NUMBER OF CPU CORES: 4


# Data Preparation

IMPORTANT:

1) The filepath may need to be changed to point to the location of the datafile.\
2) The lineterminator='\n' parameter is necessary to ensure Pandas reads the file correctly.

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load original data file for custom preprocessing in NER and Sentiment Analysis
tweets_original = pd.read_csv('/content/drive/MyDrive/Ukraine_war.csv')

# load cleaned data 
tweets = pd.read_csv('/content/drive/MyDrive/ukraine_war_clean_.csv', lineterminator='\n')

In [ ]:
# view original file
tweets_original.head()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,outlinks,tcooutlinks,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,Searh
0,snscrape.modules.twitter.Tweet,https://twitter.com/ksenijapavlovic/status/150...,2022-03-05 23:59:58+00:00,There was an anti war protest for #Ukraine as ...,There was an anti war protest for #Ukraine as ...,1500259859937337349,"{'_type': 'snscrape.modules.twitter.User', 'us...",2,4,25,0,1500259859937337349,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,['https://twitter.com/Reuters/status/150013026...,['https://t.co/CTulO2hTsA'],"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,"{'_type': 'snscrape.modules.twitter.Tweet', 'u...",NaN,NaN,NaN,NaN,NaN,['Ukraine'],NaN,Ukraine war
1,snscrape.modules.twitter.Tweet,https://twitter.com/ty_phoon31/status/15002598...,2022-03-05 23:59:58+00:00,https://t.co/Z1pURKlNHu NATO followed USA into...,abc.net.au/news/2022-03-0… NATO followed USA i...,1500259857160568834,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1500259857160568834,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,['https://www.abc.net.au/news/2022-03-05/nato-...,['https://t.co/Z1pURKlNHu'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ukraine war
2,snscrape.modules.twitter.Tweet,https://twitter.com/ann_peace22/status/1500259...,2022-03-05 23:59:54+00:00,@mfa_russia @RusEmbUSA @RussiaUN @MFA_Ukraine ...,@mfa_russia @RusEmbUSA @RussiaUN @MFA_Ukraine ...,1500259841520111616,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1500182315112321028,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,NaN,NaN,NaN,NaN,NaN,1.500182e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,['IStandWithUkraine'],NaN,Ukraine war
3,snscrape.modules.twitter.Tweet,https://twitter.com/GardensR4Health/status/150...,2022-03-05 23:59:52+00:00,@nytimes This war could have been avoided by e...,@nytimes This war could have been avoided by e...,1500259833915883525,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1499516258781437953,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN,NaN,1.499516e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,Ukraine war
4,snscrape.modules.twitter.Tweet,https://twitter.com/kbrunolieber/status/150025...,2022-03-05 23:59:52+00:00,@ggreenwald Remember the biggest mistake made ...,@ggreenwald Remember the biggest mistake made ...,1500259832649203713,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,2,0,1500195950136020993,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN,NaN,1.500200e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,Ukraine war


In [ ]:
# view cleaned data
tweets.head()

,content,date,content_clean,tokenized,tokens_no_stopwords,token_sents_no_stops,lemmatized_no_stops,lemmatized_sent_no_stops
0,There was an anti war protest for #Ukraine as ...,2022-03-05 23:59:58+00:00,there was an anti war protest for ukraine as w...,"['there', 'was', 'an', 'anti', 'war', 'protest...","['anti', 'war', 'protest', 'ukraine', 'well', ...",anti war protest ukraine well getting headline...,"['anti', 'war', 'protest', 'ukraine', 'well', ...",anti war protest ukraine well headline serbia ...
1,https://t.co/Z1pURKlNHu NATO followed USA into...,2022-03-05 23:59:58+00:00,nato followed usa into afghanistan forgotten a...,"['nato', 'followed', 'usa', 'into', 'afghanist...","['nato', 'followed', 'usa', 'afghanistan', 'fo...",nato followed usa afghanistan forgotten alread...,"['nato', 'follow', 'usa', 'afghanistan', 'forg...",nato follow usa afghanistan forget already tri...
2,@mfa_russia @RusEmbUSA @RussiaUN @MFA_Ukraine ...,2022-03-05 23:59:54+00:00,it s not russophobia we just hate putin and al...,"['it', 's', 'not', 'russophobia', 'we', 'just'...","['russophobia', 'hate', 'putin', 'enablers', '...",russophobia hate putin enablers russian people...,"['russophobia', 'hate', 'putin', 'enablers', '...",russophobia hate putin enablers russian people...
3,@nytimes This war could have been avoided by e...,2022-03-05 23:59:52+00:00,this war could have been avoided by ensuring t...,"['this', 'war', 'could', 'have', 'been', 'avoi...","['war', 'could', 'avoided', 'ensuring', 'putin...",war could avoided ensuring putin ukraine could...,"['war', 'could', 'avoid', 'ensure', 'putin', '...",war could avoid ensure putin ukraine could joi...
4,@ggreenwald Remember the biggest mistake made ...,2022-03-05 23:59:52+00:00,remember the biggest mistake made by those who...,"['remember', 'the', 'biggest', 'mistake', 'mad...","['remember', 'biggest', 'mistake', 'made', 'su...",remember biggest mistake made supported afghan...,"['remember', 'big', 'mistake', 'make', 'suppor...",remember big mistake make support afghanistan ...


In [ ]:
# convert to datetime object
tweets.date = pd.to_datetime(tweets.date)

In [ ]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199480 entries, 0 to 199479
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   content                   199480 non-null  object             
 1   date                      199480 non-null  datetime64[ns, UTC]
 2   content_clean             199480 non-null  object             
 3   tokenized                 199480 non-null  object             
 4   tokens_no_stopwords       199480 non-null  object             
 5   token_sents_no_stops      199480 non-null  object             
 6   lemmatized_no_stops       199480 non-null  object             
 7   lemmatized_sent_no_stops  199480 non-null  object             
dtypes: datetime64[ns, UTC](1), object(7)
memory usage: 12.2+ MB


### Group Tweets by Time

In [ ]:
# get range of dates in corpus
last_date = tweets.date.max().date() + timedelta(days=1)
first_date = tweets.date.min().date()
date_range = pd.date_range(start=first_date, end=last_date)
date_range

DatetimeIndex(['2021-12-31', '2022-01-01', '2022-01-02', '2022-01-03',
               '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
               '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11',
               '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15',
               '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19',
               '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23',
               '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27',
               '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31',
               '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04',
               '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08',
               '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12',
               '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16',
               '2022-02-17', '2022-02-18', '2022-02-19', '2022-02-20',
               '2022-02-21', '2022-02-22', '2022-02-23', '2022-02-24',
      

In [ ]:
# function to group tweets by selected time interval
def tweets_by_time_interval(data=None, interval=2, just_groups=False):
  """
  Parameters:
    data: dataframe (tweets)
    interval: int (number of days in each time interval)
    just_groups: bool (return only the groupings of dates)

  Returns:
    date_groups: list
    tweets_date: dataframe
  """
  selected_interval = interval
  date_groups = list(zip(*(iter(date_range),) * selected_interval))
  if just_groups:
    return date_groups
  tweets_date = []
  for i in date_groups:
    start = str(i[0].to_pydatetime())
    end = str(i[-1].to_pydatetime() + timedelta(days=1))
    tweets_date.append(data.loc[(data.date > start) & (data.date <= end)])
  return tweets_date

In [ ]:
# define function to remove emojis
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

--- 
---
# Part 2. Event Summarization and Performance Evaluation

## 2.1. Abstractive Text Summarization

### 2.1.1. Custom data preparation

In [ ]:
tweets_ = tweets.copy()

In [ ]:
# convert to datetime object
tweets_.date = pd.to_datetime(tweets_['date']).dt.date

In [ ]:
# tweet counts
tweets_counts = tweets_.groupby(['date']).size().reset_index(name='counts')

In [ ]:
# defining a lambda function to combine tweets
combine_tweets = lambda text: ".".join(text)

In [ ]:
# combining tweets for same day
combined_tweets = tweets_.groupby(by='date').agg({'content_clean': combine_tweets}).reset_index()
combined_tweets.head()

,date,content_clean
0,2021-12-31,the us want russia in ukraine so they can fight them in a proxy war they will supply ukraine equipment maybe thru other nations and watch russia get stuck into a grind similar to vietnam or afghanistan a serious war like that will shatter the russian military.after the cold war the usa made guarantees to russia that nato wouldn t expand to eastern europe can t blame russia for drawing a line at ukraine.nato really wants to lure russia to war with ukraine nato should stop provoking russia won t attack first.if putin moves in ukraine he wins but at a cost to russia future ukraine will fight into a quagmire and russia doesn t have the youth population to replace their army after a bloody vietnam afghanistan type war.look forward to seeing you propagandists end up in the docket in the hague on war crimes charges.i didn t know that the term european theater was still in use lol i mean there s the ukraine russia war but still.happy new year s ukraine i want you to know i still love you a...
1,2022-01-01,what is the purpose of whitewashing russian bellicose stance justifying russian threats all while it is russia that wages war of agression in ukraine which has already taken more than lives and forced millions of ukrainians to become internally displaced fleeing the war.what is the purpose of mental gymnastics to position russia as a threatened side again it was russia that started the war in ukraine and continues attacks in ukrainian military from russian occupied ukrainian territory.war drums uss harry s truman carrier group ordered to stay in mediterranean as tensions rise between russia ukraine.what happened next both nazi germany and ussr as allies simultaneously invaded poland and started ww now we have a situation when russia invaded ukraine in and wages war of aggression on ukraine for years noone else is willing to invade ukraine why the false analogies.do the separatists in ukraine count what is their problem anyway why do they want autonomy do they have russian citizensh...
2,2022-01-02,hilarious kremlin propaganda bullshit i could start tweeting photos of russian nazi lovers what s the point the fools the fools the fools if you support the kremlin in its war against ukraine you support imperialism and as such british imperialism in ireland.biden vows us to act decisively if russia invades ukraine what s the meaning of biden s statement the us to act decisively if russia invade ukraine hopefully biden isn t going to get us into a nuclear war with russia because of ukraine or anyone else other than our own security.actually decentamericans are fully behind drawing a red line when it comes to putin we know they they have been waging a stealth war against us with your super willing help treasonweasel amp a real one against ukraine we owe them for standing up to tfg.hopefully we the people see through this war spin the deception and not hand them our vote russian collusion ukraine quid pro quo now ukraine invasion on and on.never underestimate the stupidity of the hum...
3,2022-01-03,imagine putin xi and the mullahs simultaneously making their moves the biden administration and our woke military will lose a multi theater war tough to be ukraine skorea taiwan or iraq.the whole article is interesting even if the part on cyber is a bit hand wavy but particularly intriguing is the discussion of the armenia azerbaijan war and cheap drones bear meet porcupine unconventional deterrence for ukraine via.putin is trying to establish a new cold war our sanctions have not been enough to deter russia if putin invades ukraine it will be bloody it seems like putin is deadset on it.so finland has joined the queue along with countries using ukraine as a pretext to join nato guess there are benefits if war crimes and genocides are the tools of your trade i guess we ll need russia and china more than ever now to counter us nato opportunism and aggression.i don t do predictions but this is facts remember when the 

In [ ]:
combined_tweets['date'] = pd.to_datetime(combined_tweets['date'], format='%Y-%m-%d')
tweets_counts['date'] =  pd.to_datetime(tweets_counts['date'],format='%Y-%m-%d')
combined_df = pd.merge(combined_tweets,tweets_counts,on='date')

### 2.1.2. Using T5 Transformers

In [ ]:
summarizer = pipeline("summarization", model="t5-small", tokenizer='t5-base', framework='tf')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
def get_transformer_summary(cleaned_tweets):
    preprocess_text = str(cleaned_tweets)
    summary = summarizer(preprocess_text, max_length=300, min_length=10)
    return summary

In [ ]:
# get the summary of tweets of a requested date.
def tweets_against_dates(given_date,functionName):
  combined_df['date'] = pd.to_datetime(combined_df['date'], format='%Y-%m-%d')
  sampled_tweets = combined_df.loc[combined_df['date']==given_date]
  sampled_tweets['content_clean'] = sampled_tweets['content_clean'].values[0][:1500]
  sampled_tweets['trans_summary'] = sampled_tweets['content_clean'].apply(functionName)
  sampled_tweets.reset_index(inplace=True, drop=True)
  return sampled_tweets

In [ ]:
dated_tweets = tweets_against_dates('2022-01-02',get_transformer_summary)

In [ ]:
pd.set_option('max_colwidth', 10000)
dated_tweets

,date,content_clean,counts,trans_summary
0,2022-01-02,hilarious kremlin propaganda bullshit i could start tweeting photos of russian nazi lovers what s the point the fools the fools the fools if you support the kremlin in its war against ukraine you support imperialism and as such british imperialism in ireland.biden vows us to act decisively if russia invades ukraine what s the meaning of biden s statement the us to act decisively if russia invade ukraine hopefully biden isn t going to get us into a nuclear war with russia because of ukraine or anyone else other than our own security.actually decentamericans are fully behind drawing a red line when it comes to putin we know they they have been waging a stealth war against us with your super willing help treasonweasel amp a real one against ukraine we owe them for standing up to tfg.hopefully we the people see through this war spin the deception and not hand them our vote russian collusion ukraine quid pro quo now ukraine invasion on and on.never underestimate the stupidity of the human race with a weak amp demented a coordinated russian invasion of ukraine amp a chinese invasion of taiwan can easily happen war is always a pretext for economic resets russia holds more gold than that is very telling.decisively explain that mr biden draw us a picture you tired of not having troops in a war there is no stomach in america for anything like a war but it would be good for business sorry ukraine either way you get trounced.putin will not invade ukraine but to stifle it creating a tense,286,[{'summary_text': 'kremlin propaganda bullshit i could start tweeting photos of nazi lovers .biden vows us to act decisively if russia invades ukraine . mr biden s statement the us will not get us into a nuclear war with russia .'}]


### 2.2.3. Using BART

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
def BART_summary(ARTICLE):
  print(ARTICLE)
  input_tokens = tokenizer.batch_encode_plus([ARTICLE], return_tensors='pt', max_length=1024,truncation=True)['input_ids']
  encoded_ids = model.generate(input_tokens,num_beams=4,length_penalty=2.0,max_length=150,min_length=50,no_repeat_ngram_size=3)
  summary = tokenizer.decode(encoded_ids.squeeze(), skip_special_tokens=True)
  summ = textwrap.fill(summary, 100)
  summ = summ.strip().replace("\n","")
  return summ

In [ ]:
dated_tweets_BART = tweets_against_dates('2022-01-02',BART_summary)

hilarious kremlin propaganda bullshit i could start tweeting photos of russian nazi lovers what s the point the fools the fools the fools if you support the kremlin in its war against ukraine you support imperialism and as such british imperialism in ireland.biden vows us to act decisively if russia invades ukraine what s the meaning of biden s statement the us to act decisively if russia invade ukraine hopefully biden isn t going to get us into a nuclear war with russia because of ukraine or anyone else other than our own security.actually decentamericans are fully behind drawing a red line when it comes to putin we know they they have been waging a stealth war against us with your super willing help treasonweasel amp a real one against ukraine we owe them for standing up to tfg.hopefully we the people see through this war spin the deception and not hand them our vote russian collusion ukraine quid pro quo now ukraine invasion on and on.never underestimate the stupidity of the human r

In [ ]:
pd.set_option('max_colwidth', 10000)
dated_tweets_BART.head()

,date,content_clean,counts,trans_summary
0,2022-01-02,hilarious kremlin propaganda bullshit i could start tweeting photos of russian nazi lovers what s the point the fools the fools the fools if you support the kremlin in its war against ukraine you support imperialism and as such british imperialism in ireland.biden vows us to act decisively if russia invades ukraine what s the meaning of biden s statement the us to act decisively if russia invade ukraine hopefully biden isn t going to get us into a nuclear war with russia because of ukraine or anyone else other than our own security.actually decentamericans are fully behind drawing a red line when it comes to putin we know they they have been waging a stealth war against us with your super willing help treasonweasel amp a real one against ukraine we owe them for standing up to tfg.hopefully we the people see through this war spin the deception and not hand them our vote russian collusion ukraine quid pro quo now ukraine invasion on and on.never underestimate the stupidity of the human race with a weak amp demented a coordinated russian invasion of ukraine amp a chinese invasion of taiwan can easily happen war is always a pretext for economic resets russia holds more gold than that is very telling.decisively explain that mr biden draw us a picture you tired of not having troops in a war there is no stomach in america for anything like a war but it would be good for business sorry ukraine either way you get trounced.putin will not invade ukraine but to stifle it creating a tense,286,biden vows us to act decisively if russia invades ukraine. decentamericans are fully behind drawinga red line when it comes to putin. we know they they have been waging a stealth war against us. weowe them for standing up to tfg.


## 2.2. Extractive Text Summarization

### 2.2.1. Custom data preparation


In [ ]:
tweets_0 = tweets_original.copy()
# convert to datetime object
tweets_0['date'] =  pd.to_datetime(tweets_0['date'],format='%Y-%m-%d')
tweets_0.date = pd.to_datetime(tweets_0['date']).dt.date
gen_df = tweets_0.groupby('date', as_index=False)['content'].apply('.'.join)
gen_df.head(1)

date  \
0  2021-12-31   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
# clean tweets

def genism_clean_text(corpus):    
    corpus = [re.sub(r'@\w+','',tweet) for tweet in corpus]
        
    # remove amp 
    corpus = [re.sub(r'&amp','',tweet) for tweet in corpus]
    # remove hashtag symbols
    corpus = [re.sub(r'#','',tweet) for tweet in corpus]
    # remove twitter handles
    corpus = [re.sub(r'@[A-Za-z0-9_]+','',tweet) for tweet in corpus]
    # remove urls
    corpus = [re.sub(r'http\S+','',tweet) for tweet in corpus]  
    return corpus

gen_df['raw_content'] = genism_clean_text(gen_df.content)
# remove emojis
gen_df['raw_content']= gen_df['raw_content'].apply(lambda x: remove_emoji(x))
gen_df.head(1)

date  \
0  2021-12-31   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

### 2.2.2. Gensim Summary

In [ ]:
def genism_summary(cleaned_tweets):
  summ = summarize(cleaned_tweets, word_count=100)
  summ = summ.strip().replace("\n","")
  return summ

In [ ]:
def tweets_against_dates_gen(given_date,functionName):
  # print(functionName)
  gen_df['date'] = pd.to_datetime(gen_df['date'], format='%Y-%m-%d')
  sampled_tweets = gen_df.loc[gen_df['date']==given_date]
  sampled_tweets['trans_summary'] = sampled_tweets['raw_content'].apply(functionName)
  return sampled_tweets

In [ ]:
dated_tweets_genism = tweets_against_dates_gen('2022-01-02',genism_summary)
dated_tweets_genism.head()

date  \
2 2022-01-02   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### 2.2.3. TextRank - using SUMY

In [ ]:
def sumy_textRank_summarizer(cleaned_text):
    parser =PlaintextParser.from_string(cleaned_text,Tokenizer("english"))
    TR_summarizer = TextRankSummarizer()
    summary =TR_summarizer(parser.document,2)
    summary_list = [str(sentence) for sentence in summary]
    result = ' '.join(summary_list)
    return result

In [ ]:
dated_tweets_sumy_TR = tweets_against_dates_gen('2022-01-02',sumy_textRank_summarizer)
dated_tweets_sumy_TR.head()

date  \
2 2022-01-02   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### 2.2.4. LEX RANK - using SUMY

LexRank is another graph-based summarization approach. It's comparable to TextRank in that it's unsupervised. Between two sentences, LexRank employs Cosine similarity rather than Jaccard. For all of the sentences in the document, this similarity score will be utilized to create a weighted graph. LexRank also ensures that the topmost sentences aren't too similar to one another. LexRank is a graph-based algorithm that is simple to comprehend and use. Its outcomes aren't as semantic.

In [ ]:
def sumy_Lex_summarizer(cleaned_text):
    parser =PlaintextParser.from_string(cleaned_text,Tokenizer("english"))
    lex_summarizer = LexRankSummarizer()
    summary = lex_summarizer(parser.document,3)
    summary_list = [str(sentence) for sentence in summary]
    result = ' '.join(summary_list)
    return result

In [ ]:
dated_tweets_sumy_lex = tweets_against_dates_gen('2022-01-02',sumy_Lex_summarizer)
dated_tweets_sumy_lex.head()

date  \
2 2022-01-02   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### 2.2.5. LSA - using SUMY

Latent Semantic Analysis is based on Singular value decomposition(SVD). It compresses the data into a lower-dimensional space. It accomplishes spatial decomposition and collects data in a single vector, with the amplitude of that singular vector indicating the relevance. LSA can extract semantically linked sentences, but its computation is time-consuming.

In [ ]:
def sumy_LSA_summarizer(cleaned_text):
    parser =PlaintextParser.from_string(cleaned_text,Tokenizer("english"))
    summarizer_lsa = LsaSummarizer()
    summary = summarizer_lsa(parser.document,3)
    summary_list = [str(sentence) for sentence in summary]
    result = ' '.join(summary_list)
    return result

In [ ]:
dated_tweets_sumy_LSA = tweets_against_dates_gen('2022-01-02',sumy_LSA_summarizer)
dated_tweets_sumy_LSA.head()

date  \
2 2022-01-02   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### 2.2.6. Using SpaCy

In [ ]:
def spacy_summarizer(text):
    per = 0.005
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    summary = summary.strip().replace("\n","")
    return summary

In [ ]:
dated_tweets_spacy = tweets_against_dates_gen('2022-01-02',spacy_summarizer)
dated_tweets_spacy.head()

date  \
2 2022-01-02   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### 2.2.7. Using textrank4zh

In [ ]:
summaryList = list()
def TextRank4_summary(cleaned_text):
  tr4s = TextRank4Sentence()
  tr4s.analyze(text=cleaned_text, lower=True, source = 'all_filters')
  for item in tr4s.get_key_sentences(num=3):
    summaryList.append(item.sentence)
    summary=''.join(summaryList)
    summary = summary.strip().replace("\n","")

  return summary

In [ ]:
dated_tweets_TextRank4 = tweets_against_dates_gen('2022-01-02',TextRank4_summary)
dated_tweets_TextRank4.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.053 seconds.
Prefix dict has been built successfully.


date  \
2 2022-01-02   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### 2.2.8. Measuring Overall Text Difficulty

In [ ]:
def check_difficulty(cleaned_text):
  difficulty = textstat.flesch_reading_ease(cleaned_text)
  read_time = textstat.reading_time(cleaned_text, ms_per_char=70)
  count = textstat.lexicon_count(cleaned_text, removepunct=True)
  return difficulty,read_time,count

In [ ]:
sampled_df = gen_df[gen_df['date']=='2022-01-02']
measuringList =sampled_df['raw_content'].apply(check_difficulty)
measuringList= measuringList.tolist()
print('Text difficulty -->',measuringList[0][0])
print('Text Read Time -->',measuringList[0][1])
print('Text Count word -->',measuringList[0][2])

Text difficulty --> 74.08
Text Read Time --> 3686.34
Text Count word --> 10599


Flesch reading ease score:
- 90-100 | Very Easy
- 80-89  | Easy
- 70-79  | Fairly Easy
- 60-69  | Standard
- 50-59  | Fairly Difficult
- 30-49  | Difficult
- <30    | Very Confusing